In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
from IPython.display import display

# Load data
data_path = 'data.xlsx'  # Update with your file path
data = pd.read_excel(data_path)

# Display the first few rows of the data
display(data.head())

# Select relevant features and the target variable
features = [
    'status_kemahasiswaan', 'pernah_ikut_mbmk', 'pernah_mbkm_apapun', 
    'performa_ipk', 'nilai_ipk', 'ikut_organisasi', 'jumlah_organisasi', 
    'scan_ktp', 'upload_sertifikat', 'upload_cv', 'upload_surat_rekomendasi'
]
target = 'lolos_mbkm'

# Convert all values in the columns to strings to handle mixed types
data[features] = data[features].astype(str)

# Encode categorical variables
label_encoders = {}
for col in features + [target]:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Save encoders
joblib.dump(label_encoders, 'label_encoders.pkl')

# Separate features and target variable
X = data[features]
y = data[target]

# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save scaler
joblib.dump(scaler, 'scaler.pkl')

print("Data preprocessing complete. Encoders and scaler saved.")


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Train the SVM model
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Save the trained model
joblib.dump(svm_model, 'svm_model_mbkm.pkl')

print("Model training complete. Model saved.")


In [ ]:
# Train the SVM model
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Make predictions
y_pred = svm_model.predict(X_test)

y_pred

In [ ]:
# Evaluate the model
y_pred = svm_model.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nAccuracy Score:")
print(accuracy_score(y_test, y_pred))


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Visualize the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib

# Load the dummy test data
test_data_path = 'test.xlsx'  # Update with your file path
test_data = pd.read_excel(test_data_path)

# Extract original data for later combination
original_test_data = test_data[['nama', 'nim', 'jurusan', 'status_kemahasiswaan']].copy()

# Drop columns that are not features
test_data = test_data.drop(columns=['nama', 'nim', 'jurusan'])

# Convert all values in the columns to strings to handle mixed types
test_data = test_data.astype(str)

# Load encoders and scaler
label_encoders = joblib.load('label_encoders.pkl')  # Update with your file path
scaler = joblib.load('scaler.pkl')  # Update with your file path

# Encode categorical variables using the same label encoders used in training
encoded_test_data = test_data.copy()
for col in encoded_test_data.columns:
    if col in label_encoders:
        le = label_encoders[col]
        # Handle unseen labels by replacing with the most common class or a specific class
        encoded_test_data[col] = encoded_test_data[col].apply(
            lambda x: le.transform([x])[0] if x in le.classes_ else le.transform([le.classes_[0]])[0]
        )

# Separate data into detected and undetected
detected_data_mask = encoded_test_data.apply(lambda x: x != 'Tidak Terdeteksi').all(axis=1)
undetected_data_mask = ~detected_data_mask

detected_test_data = encoded_test_data[detected_data_mask]
undetected_test_data = encoded_test_data[undetected_data_mask]

# Ensure all necessary features are present in detected test data
for feature in scaler.feature_names_in_:
    if feature not in detected_test_data.columns:
        detected_test_data[feature] = 0

# Reorder columns to match the order used in training
detected_test_data = detected_test_data[scaler.feature_names_in_]

# Normalize the detected data using the same scaler used in training
if not detected_test_data.empty:
    detected_test_data_scaled = scaler.transform(detected_test_data)
    # Load the trained model
    model_filename = 'svm_model_mbkm.pkl'  # Update with your file path
    svm_model = joblib.load(model_filename)
    # Make predictions
    predictions = svm_model.predict(detected_test_data_scaled)

    # Print SVM predictions
    # print("SVM Predictions:", predictions)

    detected_test_data['lolos_mbkm'] = predictions
    # Decode the predictions to original labels
    detected_test_data['lolos_mbkm'] = detected_test_data['lolos_mbkm'].apply(lambda x: 'Lolos' if x == 1 else 'Tidak Lolos')

# Add 'Tidak Terdeteksi' label to undetected data``
undetected_test_data['lolos_mbkm'] = 'Tidak Terdeteksi'

# Combine detected and undetected data
combined_data = pd.concat([detected_test_data, undetected_test_data], axis=0)

# Combine with original data for output
output_data = pd.concat([original_test_data.reset_index(drop=True), combined_data['lolos_mbkm'].reset_index(drop=True)], axis=1)

# Display the results
display(output_data.head())

# Save the test data with predictions
test_data_with_predictions_path = 'hasil.xlsx'  # Update with your file path
output_data.to_excel(test_data_with_predictions_path, index=False)

print(f"Predictions saved to {test_data_with_predictions_path}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the results data
output_data_path = 'hasil.xlsx'
output_data = pd.read_excel(output_data_path)

# Load the original test data
original_test_data_path = 'test.xlsx'
original_test_data = pd.read_excel(original_test_data_path)

# Display the first few rows of the output data
display(output_data.head())

# List to store skipped plots
skipped_plots = []

# Function to plot the count of categories
def plot_count(data, column, title):
    try:
        plt.figure(figsize=(8, 4))
        category_mapping = {category: i for i, category in enumerate(data[column].unique(), start=1)}
        data['category_number'] = data[column].map(category_mapping)
        ax = sns.countplot(x='category_number', data=data)
        plt.title(title)
        
        # Create a custom legend
        handles = [plt.Line2D([0], [0], marker='o', color=ax.patches[i].get_facecolor(), linestyle='') 
                   for i, category in enumerate(category_mapping)]
        labels = [f"{num}: {cat}" for cat, num in category_mapping.items()]
        
        plt.legend(handles=handles, title=column, loc='center left', bbox_to_anchor=(1, 0.5), labels=labels)
        plt.xlabel('Category Number')
        plt.show()

        # Display the table
        if column in data.columns:
            count_data = data[[column, 'lolos_mbkm']].value_counts().unstack().fillna(0)
            display(count_data)
        else:
            raise ValueError(f"No data available for {column}")
    except Exception as e:
        skipped_plots.append(f"{title}: {e}")

# Add missing columns from the original data if necessary
columns_to_add = ['status_kemahasiswaan', 'jenis_kelamin']
for col in columns_to_add:
    if col not in output_data.columns and col in original_test_data.columns:
        output_data[col] = original_test_data[col]

# Plot the total counts
print("Total Lolos MBKM")
plot_count(output_data, 'lolos_mbkm', 'Total Lolos MBKM')

# Plot the counts by gender (if available)
if 'jenis_kelamin' in output_data.columns:
    print("Lolos MBKM Berdasarkan Jenis Kelamin")
    plot_count(output_data, 'jenis_kelamin', 'Lolos MBKM Berdasarkan Jenis Kelamin')

# Plot the counts by jurusan
print("Lolos MBKM Berdasarkan Jurusan")
plot_count(output_data, 'jurusan', 'Lolos MBKM Berdasarkan Jurusan')

# Plot the counts by status kemahasiswaan (if available)
if 'status_kemahasiswaan' in output_data.columns:
    print("Lolos MBKM Berdasarkan Status Kemahasiswaan")
    plot_count(output_data, 'status_kemahasiswaan', 'Lolos MBKM Berdasarkan Status Kemahasiswaan')

# Plot the counts of Lolos and Tidak Lolos by gender (if gender data available)
if 'jenis_kelamin' in output_data.columns:
    print("Lolos/Tidak Lolos Berdasarkan Jenis Kelamin")
    try:
        plt.figure(figsize=(8, 4))
        category_mapping = {category: i for i, category in enumerate(output_data['jenis_kelamin'].unique(), start=1)}
        output_data['category_number'] = output_data['jenis_kelamin'].map(category_mapping)
        ax = sns.countplot(x='category_number', hue='lolos_mbkm', data=output_data)
        plt.title('Lolos/Tidak Lolos Berdasarkan Jenis Kelamin')
        
        # Create a custom legend
        handles = [plt.Line2D([0], [0], marker='o', color=ax.patches[i].get_facecolor(), linestyle='') 
                   for i, category in enumerate(category_mapping)]
        labels = [f"{num}: {cat}" for cat, num in category_mapping.items()]
        
        plt.legend(handles=handles, title='Jenis Kelamin', loc='center left', bbox_to_anchor=(1, 0.5), labels=labels)
        plt.xlabel('Category Number')
        plt.show()
        
        count_data = output_data[['jenis_kelamin', 'lolos_mbkm']].value_counts().unstack().fillna(0)
        display(count_data)
    except Exception as e:
        skipped_plots.append(f"Lolos/Tidak Lolos Berdasarkan Jenis Kelamin: {e}")

# Plot the counts of Lolos and Tidak Lolos by jurusan
print("Lolos/Tidak Lolos Berdasarkan Jurusan")
try:
    plt.figure(figsize=(8, 4))
    category_mapping = {category: i for i, category in enumerate(output_data['jurusan'].unique(), start=1)}
    output_data['category_number'] = output_data['jurusan'].map(category_mapping)
    ax = sns.countplot(x='category_number', hue='lolos_mbkm', data=output_data)
    plt.title('Lolos/Tidak Lolos Berdasarkan Jurusan')
    
    # Create a custom legend
    handles = [plt.Line2D([0], [0], marker='o', color=ax.patches[i].get_facecolor(), linestyle='') 
               for i, category in enumerate(category_mapping)]
    labels = [f"{num}: {cat}" for cat, num in category_mapping.items()]
    
    plt.legend(handles=handles, title='Jurusan', loc='center left', bbox_to_anchor=(1, 0.5), labels=labels)
    plt.xlabel('Category Number')
    plt.show()
    
    count_data = output_data[['jurusan', 'lolos_mbkm']].value_counts().unstack().fillna(0)
    display(count_data)
except Exception as e:
    skipped_plots.append(f"Lolos/Tidak Lolos Berdasarkan Jurusan: {e}")

# Print skipped plots
if skipped_plots:
    print("\nSkipped Plots:")
    for plot in skipped_plots:
        print(plot)
